In [1]:
import os

from dotenv import load_dotenv
from pydantic import BaseModel, Field
from typing import List
from typing_extensions import Annotated, Literal, TypedDict

from langgraph.graph import add_messages


_ = load_dotenv()
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY", "")

class Router(BaseModel):
    """Analyze the unread email and route it according to its content.
    """
    reasoning: str = Field(
        description="Step-by-step reasoning behind the classification."
    )
    classification: Literal["ignore", "respond", "notify"] = Field(
        description="The classification of an email: 'ignore' for irrelevant emails, 'notify' for important information that doesn't need a response, 'respond' for emails that need a reply",
    )

class State(TypedDict):
    email_input: str
    messages: Annotated[List, add_messages]
    
profile = {
    "name": "Yoona",
    "full_name": "Lim Yoona",
    "user_profile_background": "Senior software engineer leading a team of 5 developers",
}

In [2]:
from typing import List

from langchain_core.tools import tool
from langchain_openai import OpenAIEmbeddings

from langgraph.store.memory import InMemoryStore

from langmem import create_manage_memory_tool, create_search_memory_tool


@tool
def write_email(to: str, subject: str, content: str) -> str:
    """Write and send an email."""
    # placeholder response - in real app would send email
    return f"Email sent to {to} with subject '{subject}'."

@tool
def schedule_meeting(
    attendees: List[str], 
    subject: str, 
    duration_minutes: int, 
    preferred_day: str
) -> str:
    """Schedule a calendar meeting."""
    # placeholder response - in real app would check calendar and schedule
    return f"Meeting '{subject}' scheduled for {preferred_day} with {len(attendees)} attendees."

@tool
def check_calendar_availability(day: str) -> str:
    """Check calendar availability for a given day."""
    # placeholder response - in real app would check actual calendar
    return f"Available times on {day}: 9:00 AM, 2:00 PM, 4:00 PM."

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
)

store = InMemoryStore(
    index={"embed": embeddings}
)

manage_memory_tool = create_manage_memory_tool(
    namespace=(
        "email_assistant", 
        "{langgraph_user_id}",
        "collection"
    )
)
search_memory_tool = create_search_memory_tool(
    namespace=(
        "email_assistant",
        "{langgraph_user_id}",
        "collection"
    )
)

In [3]:
from prompts import agent_system_prompt_memory_2, prompt_instructions


def create_prompt(state):
    return [
        {
            "role": "system", 
            "content": agent_system_prompt_memory_2.format(
                full_name=profile["full_name"],
                name=profile["name"],
				instructions=prompt_instructions["agent_instructions"], 
            )
        }
    ] + state['messages']

In [4]:
from langgraph.prebuilt import create_react_agent

from langchain_openai import ChatOpenAI


llm = ChatOpenAI(
	model="openai/gpt-4.1",
	base_url="https://openrouter.ai/api/v1",
	api_key=OPENROUTER_API_KEY
)

tools= [
    write_email, 
    schedule_meeting,
    check_calendar_availability,
    manage_memory_tool,
    search_memory_tool
]

response_agent = create_react_agent(
    llm,
    tools=tools,
    prompt=create_prompt,
    store=store	    # use this to ensure the store is passed to the agent 
)

In [5]:
# template for formating an example to put in prompt
template = """
	Email Subject: {subject}
	Email From: {from_email}
	Email To: {to_email}
	Email Content: 
	'''
	{content}
	'''
	> Triage Result: {result}
"""

# format list of few shots
def format_few_shot_examples(examples):
    strs = ["Here are some previous examples:"]
    for eg in examples:
        strs.append(
            template.format(
                subject=eg.value["email"]["subject"],
                to_email=eg.value["email"]["to"],
                from_email=eg.value["email"]["author"],
                content=eg.value["email"]["email_thread"][:400],
                result=eg.value["label"],
            )
        )
    return "\n\n------------\n\n".join(strs)

In [6]:
from typing import Literal

from langchain_openai import ChatOpenAI

from langgraph.graph import StateGraph, START, END
from langgraph.types import Command

from prompts import triage_system_prompt, triage_user_prompt, prompt_instructions


llm = ChatOpenAI(
	model="openai/gpt-4.1",
	base_url="https://openrouter.ai/api/v1",
	api_key=OPENROUTER_API_KEY
)
llm_router = llm.with_structured_output(Router)

def triage_router(state: State, config, store) -> Command[Literal["response_agent", "__end__"]]:
	author = state['email_input']['author']
	to = state['email_input']['to']
	subject = state['email_input']['subject']
	email_thread = state['email_input']['email_thread']

	namespace = ("email_assistant", config['configurable']['langgraph_user_id'], "examples")
	examples = store.search(
		namespace, 
		query=str({"email": state['email_input']})
	) 
	examples=format_few_shot_examples(examples)

	system_prompt = triage_system_prompt.format(
		full_name=profile["full_name"],
		name=profile["name"],
		user_profile_background=profile["user_profile_background"],
		triage_no=prompt_instructions["triage_rules"]["ignore"],
		triage_notify=prompt_instructions["triage_rules"]["notify"],
		triage_email=prompt_instructions["triage_rules"]["respond"],
		examples=examples
	)
	user_prompt = triage_user_prompt.format(
		author=author, 
		to=to, 
		subject=subject, 
		email_thread=email_thread
	)
	result = llm_router.invoke(
		[
			{"role": "system", "content": system_prompt},
			{"role": "user", "content": user_prompt},
		]
	)
	if result.classification == "respond":
		print("📧 Classification: RESPOND - This email requires a response")
		goto = "response_agent"
		update = {
			"messages": [
				{
					"role": "user",
					"content": f"Respond to the email {state['email_input']}",
				}
			]
		}
	elif result.classification == "ignore":
		print("🚫 Classification: IGNORE - This email can be safely ignored")
		update = None
		goto = END
	elif result.classification == "notify":
		# if real life, this would do something else
		print("🔔 Classification: NOTIFY - This email contains important information")
		update = None
		goto = END
	else:
		raise ValueError(f"Invalid classification: {result.classification}")
	return Command(goto=goto, update=update)

email_agent = StateGraph(State)
email_agent = email_agent.add_node(triage_router)
email_agent = email_agent.add_node("response_agent", response_agent)
email_agent = email_agent.add_edge(START, "triage_router")
email_agent = email_agent.compile(store=store)

In [7]:
data = {
    "email": {
		"author": "Tom Jones <tome.jones@bar.com>",
		"to": "Lim Yoona <yoona@company.com>",
		"subject": "Quick question about API documentation",
		"email_thread": """
        	Hi Yoon - want to buy documentation?
        """,
	},
    "label": "ignore"
}

import uuid

store.put(
    ("email_assistant", "asdfg", "examples"),
    str(uuid.uuid4()),
    data
)

In [8]:
email_input = {
    "author": "Alice Smith <alice.smith@company.com>",
    "to": "Lim Yoona <yoona@company.com>",
    "subject": "Quick question about API documentation",
    "email_thread": """
    	Hi Yoon,

		I was reviewing the API documentation for the new authentication service and noticed a few endpoints seem to be missing from the specs. Could you help clarify if this was intentional or if we should update the docs?

		Specifically, I'm looking at:
		- /auth/refresh
		- /auth/validate

		Thanks!
		Alice
    """,
}

response = email_agent.invoke(
    {"email_input": email_input}, 
    config={"configurable": {"langgraph_user_id": "asdfg"}}
)
for m in response["messages"]:
    m.pretty_print()

📧 Classification: RESPOND - This email requires a response
================================ Human Message =================================

Respond to the email {'author': 'Alice Smith <alice.smith@company.com>', 'to': 'Lim Yoona <yoona@company.com>', 'subject': 'Quick question about API documentation', 'email_thread': "\n    \tHi Yoon,\n\n\t\tI was reviewing the API documentation for the new authentication service and noticed a few endpoints seem to be missing from the specs. Could you help clarify if this was intentional or if we should update the docs?\n\n\t\tSpecifically, I'm looking at:\n\t\t- /auth/refresh\n\t\t- /auth/validate\n\n\t\tThanks!\n\t\tAlice\n    "}
================================== Ai Message ==================================
Tool Calls:
  write_email (call_HbCd6QfjyYrLzugyGc65qIwY)
 Call ID: call_HbCd6QfjyYrLzugyGc65qIwY
  Args:
    to: alice.smith@company.com
    subject: Re: Quick question about API documentation
    content: Hi Alice,

Thank you for catching t

In [9]:
email_input = {
    "author": "Alice Smith <alice.smith@company.com>",
    "to": "Lim Yoona <yoona@company.com>",
    "subject": "Follow up",
    "email_thread": """
    	Hi Yoon - want to buy documentation?
    """,
}

response = email_agent.invoke(
    {"email_input": email_input}, 
    config={"configurable": {"langgraph_user_id": "asdfg"}}
)
for m in response["messages"]:
    m.pretty_print()

🚫 Classification: IGNORE - This email can be safely ignored


In [10]:
store.search(('email_assistant', 'asdfg', 'collection'))

[]

In [11]:
store.search(('email_assistant', 'asdfg', 'examples'))

[Item(namespace=['email_assistant', 'asdfg', 'examples'], key='f557fa0f-aa21-4b9b-9e25-27a8b23a73bf', value={'email': {'author': 'Tom Jones <tome.jones@bar.com>', 'to': 'Lim Yoona <yoona@company.com>', 'subject': 'Quick question about API documentation', 'email_thread': '\n        \tHi Yoon - want to buy documentation?\n        '}, 'label': 'ignore'}, created_at='2025-04-16T06:05:29.695304+00:00', updated_at='2025-04-16T06:05:29.695309+00:00', score=None)]

In [ ]:
email_input = {
    "author": "Alice Smith <alice.smith@company.com>",
    "to": "Lim Yoona <yoona@company.com>",
    "subject": "Follow up",
    "email_thread": """
    	Hi Yoon - want to buy documentation?
    """,
}

response = email_agent.invoke(
    {"email_input": email_input}, 
    config={"configurable": {"langgraph_user_id": "mnbvc"}}
)
for m in response["messages"]:
    m.pretty_print()

🚫 Classification: IGNORE - This email can be safely ignored
